In [2]:
# plot_dec_tokens_pretty.py
import csv
import argparse
from collections import defaultdict
from datetime import datetime, date, timedelta


def parse_iso_z(ts: str) -> datetime:
    return datetime.fromisoformat(ts.replace("Z", "+00:00"))


def month_end(year: int, month: int) -> date:
    if month == 12:
        return date(year, 12, 31)
    return date(year, month + 1, 1) - timedelta(days=1)


def build_daily_tokens(csv_path: str, year: int, month: int):
    daily = defaultdict(int)

    with open(csv_path, newline="", encoding="utf-8-sig") as f:
        reader = csv.DictReader(f)
        for row in reader:
            dt = parse_iso_z(row["Date"])
            if dt.year == year and dt.month == month:
                daily[dt.date()] += int(float(row.get("Total Tokens") or 0))

    start = date(year, month, 1)
    end = month_end(year, month)

    days, tokens = [], []
    d = start
    while d <= end:
        days.append(d)
        tokens.append(daily.get(d, 0))
        d += timedelta(days=1)

    return days, tokens


def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("csv_path", nargs="?", default="usage-events-2026-01-07.csv")
    ap.add_argument("--year", type=int, default=2025)
    ap.add_argument("--month", type=int, default=12)
    ap.add_argument("--out", default="daily-usage-dec.html")
    args = ap.parse_args()

    days, tokens = build_daily_tokens(args.csv_path, args.year, args.month)
    total = sum(tokens)
    avg = round(total / len(tokens), 2) if tokens else 0

    try:
        import plotly.graph_objects as go
    except ModuleNotFoundError:
        raise SystemExit("Missing dependency: plotly. Install with: pip install plotly")

    bg = "#0b1220"          # page
    card = "#0f172a"        # plot area
    grid = "rgba(255,255,255,0.06)"
    line = "#7c3aed"        # purple
    fill = "rgba(124,58,237,0.22)"
    text = "#e5e7eb"
    muted = "rgba(229,231,235,0.75)"

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=days,
            y=tokens,
            mode="lines",
            line=dict(color=line, width=3, shape="spline", smoothing=1.1),
            fill="tozeroy",
            fillcolor=fill,
            hovertemplate="%{x|%Y-%m-%d}<br>Total tokens: %{y:,}<extra></extra>",
        )
    )

    fig.update_layout(
        template="plotly_dark",
        title=dict(
            text=f"Daily Usage • {args.year}-{args.month:02d} &nbsp;&nbsp; <span style='color:{muted}'>Total</span> {total:,}  <span style='color:{muted}'>Avg/day</span> {avg:,.2f}",
            x=0.02,
            xanchor="left",
            font=dict(size=18),
        ),
        paper_bgcolor=bg,
        plot_bgcolor=card,
        margin=dict(l=40, r=24, t=72, b=44),
        font=dict(family="Inter, system-ui, -apple-system, Segoe UI, Roboto", color=text),
        height=420,
        showlegend=False,
    )

    fig.update_xaxes(
        showgrid=False,
        tickformat="%b %d",
        ticks="outside",
        ticklen=6,
        tickcolor="rgba(255,255,255,0.2)",
        showline=False,
        zeroline=False,
    )
    fig.update_yaxes(
        showgrid=True,
        gridcolor=grid,
        tickformat=",",
        zeroline=False,
    )

    fig.write_html(args.out, include_plotlyjs="inline")  # 离线可打开
    print(f"Wrote: {args.out} (open it in your browser)")


if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--year YEAR] [--month MONTH] [--out OUT]
                             [csv_path]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/tanshixiang/Library/Jupyter/runtime/kernel-v3776f16cf7bb2905e05fc3fd41fa7a74b76ab8b58.json


SystemExit: 2

/opt/anaconda3/envs/django-5/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
